In [1]:
from torchvision import datasets

train_set = datasets.CIFAR10("/home/randd/Desktop/Desktop_Office/2024/February2024/FirstWeek/ActiveLearning3/", train=True, download=True)
test_set = datasets.CIFAR10("/home/randd/Desktop/Desktop_Office/2024/February2024/FirstWeek/ActiveLearning3/", train=False, download=True)


/home/randd/.local/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Files already downloaded and verified
Files already downloaded and verified


In [2]:
from torchvision.transforms import transforms

IMG_SIZE = 128

train_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)
test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)


In [5]:
from flash.image import ImageClassifier, ImageClassificationData
from flash.core.classification import Logits
from flash.image.classification.integrations.baal import (
    ActiveLearningDataModule,
    ActiveLearningLoop,
)

from baal.active import get_heuristic
import structlog

heuristic = "bald"

log = structlog.get_logger()


OSError: libcudart.so.10.2: cannot open shared object file: No such file or directory

In [4]:
class DataModule_(ImageClassificationData):
    @property
    def num_classes(self):
        return 10
    
dm = DataModule_.from_datasets(
        train_dataset=train_set,
        test_dataset=test_set,
        train_transform=train_transforms,
        test_transform=test_transforms,
        # Do not forget to set `predict_transform`,
        # this is what we will use for uncertainty estimation!
        predict_transform=test_transforms,
        batch_size=64,
    )

NameError: name 'ImageClassificationData' is not defined

In [ ]:
from torch import nn
from flash.image import ImageClassifier
from flash.core.classification import Logits
from functools import partial

head = nn.Sequential(
        nn.Linear(512, 512),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(512, 512),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(512, dm.num_classes),
)
LR = 0.001
model = ImageClassifier(
    num_classes=dm.num_classes,
    head=head,
    backbone="vgg16",
    pretrained=True,
    loss_fn=loss_fn,
    optimizer=partial(torch.optim.SGD, momentum=0.9, weight_decay=5e-4),
    learning_rate=LR,
    serializer=Logits(),  # Note the serializer to Logits to be able to estimate uncertainty.
)


In [ ]:
active_dm = ActiveLearningDataModule(
        dm,
        heuristic=get_heuristic(heuristic),
        initial_num_labels=1024,
        query_size=100,
        val_split=0.0,
    )
assert active_dm.has_test, "No test set?"

In [ ]:
from flash import Trainer
from flash.image.classification.integrations.baal import(
    ActiveLearningLoop,
)
from flash import Trainer

trainer = Trainer(max_epochs=2500)

active_learning_loop = ActiveLearningLoop(
    label_epocj_frequency=20,
    inference_iteration=20
)
active_learning_loop.connect(trainer.fit_loop)
trainer.fit_loop = active_learning_loop